In [1]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

output_notebook()

Loading BokehJS ...

# Tutorial \#3: Links with duration.

As our dataset in this tutorial, we will use a subset of the [MAWILab](http://www.fukuda-lab.org/mawilab/index.html) dataset.

MAWILab annotates traffic anomalies in the MAWI archive with four different labels

In [2]:
import pandas as pd
df = pd.read_csv('mawi_10k.csv', names=['u', 'v', 'ts', 'tf'])
df.head()

,u,v,ts,tf
0,0,1,0.000000,1.000007
1,4,5,0.000009,1.564206
2,9,10,0.000050,3.286796
3,11,4,0.000097,1.403983
4,12,4,0.000140,95.927778


As we see our time-domain is that of floating point (*real*) numbers.

In [3]:
df.shape[0]

10000

## The TemporalLinkSet

This dataset finds its representative in our library, by what is notated as a **TemporalLinkSet**, which can be defined as a collection of tuples $(u, v, t_{s}, t_{f})$.  

Here we will initiliaze a temporal-link-set of links with duration, in continuous-time.  
We know that our dataset **doesn't** contain intervals that overlap for the same combination of $u, v$, so we want to signify this to our class-constructor by setting `disjoint_intervals=True`, or ommiting it as this is it's default value.
In order to signify that we are in continuous-time, we should also set the parameter `discrete` to `False`.

Note that suffix `DF` of our class name, indicates the fact that this implementation of a temporal-link-set is based on a dataframe.

In [4]:
from stream_graph import TemporalLinkSetDF
tls = TemporalLinkSetDF(df, discrete=False)

The first operation one can think of, is calculating the number of nodes that appear inside our temporal linkset.

In [5]:
tls.nodeset.size

11844

Also we can calculate the total amount of time that temporal-links exists.

In [6]:
tls.size

73358.747739

### Measures
So in order to see which node has the biggest traffic towards other nodes in the network, we can just calculate and sort the `out-degree`.

In [7]:
DF = pd.DataFrame(list(tls.degree_of(direction='out')), columns=['user', 'out-degree'])
DF = DF.set_index('user').sort_values(by=['out-degree'], ascending=False)
DF.head(5)

,out-degree
user,
336,4700.973234
1830,2345.000000
507,1801.771148
259,1801.640849
1053,1801.538914


Inversely, to see which node is the target of the biggest traffic, we can just calculate and sort the `in-degree`.

In [8]:
DF = pd.DataFrame(list(tls.degree_of(direction='in')), columns=['user', 'in-degree']).set_index('user')
DF.sort_values(by=['in-degree'], ascending=False).head(5)

,in-degree
user,
113,6069.822806
40,6007.210109
24,3869.189795
4,3166.437573
163,2547.810301


In order to see which connections between two nodes lasted the most, what you need is known as the duration of the link. To retrieve links as being uni-directional, we should set the parameter `direction` to `both`.

In [9]:
data = [(a, b, d) for ((a, b), d) in tls.duration_of(direction='both')]
DF = pd.DataFrame(data, columns=['src', 'dst', 'duration'])#.set_index(['src', 'dst'])

And plot the *max-5*.

In [10]:
DF.sort_values(by=['duration'], ascending=False).head(5)

,src,dst,duration
60,150,151,900.894425
75,199,200,900.893651
139,374,375,900.886150
168,507,508,900.885582
173,507,520,900.885566


So for a first figure let's calculate the number of links through time.

In [11]:
t, value = zip(*((t, v) for (t, f), v in tls.m_at()))

Here `tls.m_at()` returns a sequence of changes. That is each time a new value `v` appears the time-stamp `t` where this value appears is provided. Time here is ascending and the information `f` signifies if the value is contained on exactly that time-stamp. That is because, the time-signature is continuous which translates to *bounds*.

In our case as the bounds are not provided, they are by default considered to be `both` closed. Their default value can be changed if the variable `default_closed` is set to one of `right`,  `left`, `neither`, `both`, upon initialization, or by providing an explicit column with the header name of `itype` with one of the above strings upon initialization, for each entry.

In [12]:
from bokeh.models.glyphs import Step
from bokeh.models import ColumnDataSource

p = figure(plot_width=400, plot_height=400)

# add a steps renderer
p.step(t, value, line_width=2, mode="before", color="green")

show(p)

But let's say we want to work on a subset of the linkstream. In order to do so we will use the `substream` method.  
Here we will extract the part of the final activity of the linkstream, signifying an interval in the important area from 0 to 100.

In [13]:
tls_sub = tls.substream(ts=[(0, 100)])
tls_sub.df

,u,v,ts,tf,s,f
0,0,1,0.000000,1.000007,True,True
1,14,15,0.000210,1.000210,True,True
2,32,33,0.000435,1.000435,True,True
3,34,35,0.000436,1.000436,True,True
4,49,50,0.000748,1.000748,True,True
5,51,52,0.000749,1.000749,True,True
6,55,56,0.000818,1.000818,True,True
7,70,71,0.001311,1.001311,True,True
8,32,76,0.001346,1.001346,True,True
9,82,83,0.001403,1.001403,True,True


Here `s` and `f` are the column names corresponding to whether the *left* and *right* bound of the interval is open or closed.

Notice here, that unlike the **instantaneous** case, the `filter` function **will not** return something equivalent with the `substream`, as the second takes the intersection of existing links, that are bigger than the required duration, whereas filter keeps only the links that satisfy a property (throwing away all the rest).

In [14]:
tls_sub = tls.filter(lambda k: (k[2] >= 0 and k[3] <= 100))
tls_sub.df

,u,v,ts,tf,s,f
0,0,1,0.000000,1.000007,True,True
1,4,5,0.000009,1.564206,True,True
2,9,10,0.000050,3.286796,True,True
3,11,4,0.000097,1.403983,True,True
4,12,4,0.000140,95.927778,True,True
5,13,4,0.000167,32.503637,True,True
6,14,15,0.000210,1.000210,True,True
7,16,17,0.000217,1.110317,True,True
8,9,20,0.000250,1.251636,True,True
9,21,4,0.000323,1.164242,True,True


### Creating a StreamGraph
Now let's create a stream-graph!

A StreamGraph $S=(T, V, W, E)$ is a collection of four elements:
- $T $, a time-set
- $V $, a node-set
- $W\subseteq T \times V$, a temporal-node-stream
- $E\subseteq T \times V \otimes V$, a temporal-link-stream

To do so lets extract the nodeset and the timeset of our temporal-link-set:

In [15]:
ns, ts = tls.nodeset, tls.timeset

Now we would like to extract the minimal temporal-node-set, that is for each node that appears in our the node-set, we want it to dissapear whenever a link does not exist inside the temporal-link-set and anyone else.

In [16]:
tns = tls.minimal_temporal_nodeset

In [17]:
from stream_graph import StreamGraph
sg = StreamGraph(ns, ts, tns, tls)

The equivalent of what we did is equivalent with:

In [18]:
sg = tls.as_stream_graph_minimal

### Stream-Graph metrics
A stream-graph is usefull for what is known as calculating coverage.

Coverage is the percentage a measure satisfies its complete possible value.
For example in traditional graphs, a coverage measure is edge-density, which is an indicator of how much sparse a graph is.

The first measures we will calculate in this section is the total coverage of the temporal-linkset:
$$\frac{|E|}{\sum_{uv \in V\times V}|T_{u} \cap T_{v}|}$$

In [19]:
sg.temporal_linkset_coverage

0.0013562740930292564

and the total coverage of the temporal-nodeset:
$$\frac{|W|}{|V\times T|}$$

In [20]:
sg.temporal_nodeset_coverage

0.009805863708490528

Now let's calculate and plot for each node $u$ it's time coverage"
$$\frac{|T_{u}|}{|T|}$$

In [21]:
from operator import itemgetter

us, tc = zip(*sorted(list(sg.time_coverage_node()), key=itemgetter(1), reverse=True))
x = list(range(1, len(us)+1))
p = figure()
p.vbar(x=x, top=tc, width=1)

show(p)

And we can do the same for links:
$$\frac{|T_{uv}|}{|T_{u} \cap T_{v}|}$$

In [22]:
link_coverage = sg.time_coverage_link()

In [23]:
from operator import itemgetter

links, tc = zip(*sorted(list(link_coverage), key=itemgetter(1), reverse=True))
x = list(range(1, len(links)+1))
p = figure()
p.step(x, tc, line_width=2, mode="before", color="#f46d43")

show(p)

Following the same logic, there are measures of coverage measured at a certain point in time.  
This measures are closely to the notion of density in static graphs.  
Thus we have node-coverage at time $t$:
$$\frac{|V_{t}|}{|V|}$$

In [24]:
t, v = zip(*((t, v) for (t, f), v in sg.node_coverage_at()))

In [25]:
from operator import itemgetter

p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

and we have link-coverage at time $t$:
$$\frac{|E_{t}|}{|V|^{2}}$$

In [26]:
t, v = zip(*((t, v) for (t, f), v in sg.link_coverage_at()))

In [27]:
p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

Now, for measures concerning the degree, there is `neighbor_coverage`:
$$\frac{|N(u)|}{\sum_{v\in V}{|T_{u}\cap T_{v}|}}$$

In [28]:
nc = sg.neighbor_coverage()

In [29]:
from operator import itemgetter

links, tc = zip(*sorted(list(nc), key=itemgetter(1), reverse=True))
x = list(range(1, len(links)+1))
p = figure()
p.step(x, tc, line_width=2, mode="before", color="#f46d43")

show(p)

And of-course we can have the neighbor-coverage of a node at a certain time-stamp:

$$\frac{|N_{t}(u)|}{|V|}$$

In [30]:
nc = sg.neighbor_coverage_at()

Finally we can measure the mean_degree of the stream-graph over-time:
$$\frac{|E_{t}|}{|W_{t}|}$$

In [31]:
t, v = zip(*((t, v) for (t, f), v in sg.mean_degree_at()))

In [32]:
p = figure()
p.step(t, v, line_width=2, mode="before", color="#f46d43")

show(p)

## Cliques
A clique in a classic graph is defined as a set of nodes, where there exists a link between all pairs of nodes (subgraphs of density 1).

![image info](./graph-clique.png)

In a link-stream of continuous duration, a clique is defined as subset of the timeset and a subset of the nodes, where links exist between everybody.

![image info](./link-stream-clique.png)

In [33]:
# Delta cliques inside a twenty minute period
mxc = tls.get_maximal_cliques(direction='both')

And now let's sort and print the ten tiggest cliques.

In [36]:
# Efficient: whenever an actor exists it connects with all the other actors existing
sorted_cliques = sorted(list(mxc), key=lambda x:(len(x[0]), x[1][1] - x[1][0]), reverse=True)
sorted_cliques[:10]

[(frozenset({150, 151}), (0.003036, 900.8974609999999)),
 (frozenset({199, 200}), (0.004163, 900.897814)),
 (frozenset({374, 375}), (0.008438, 900.8945880000001)),
 (frozenset({507, 508}), (0.010803, 900.896385)),
 (frozenset({507, 520}), (0.011094, 900.89666)),
 (frozenset({259, 260}), (0.005438, 900.888244)),
 (frozenset({249, 250}), (0.005296, 900.8869990000001)),
 (frozenset({4, 722}), (0.016544, 900.895617)),
 (frozenset({458, 459}), (0.009977, 900.8835619999999)),
 (frozenset({1203, 1204}), (0.031348, 900.894968))]